## 필요한 모듈 불러오기

In [20]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

import os
import pandas  as pd 
import math

## Data Prepration

<수집 데이터> 
1. 진보 ~ 보수 사이의 다양한 스펙트럼을 가진 언론사 4곳을 선정하여 재임 기간 동안의 기사 제목 - 직접 크롤링하여 선별 가공
    
    (한겨레 - 좌파, 경향신문 - 중도좌파, 국민일보 - 중도우파, 동아일보 - 우파)


2. 문재인 대통령의 국정 수행 지지도 - 한국갤럽, open source들을 분석하여 계산을 통해 txt 파일 생성


3. KNU 한국어 감성사전 - open source txt파일을 다운로드하여 data analysis 단계에 사용

1~3 데이터를 통합하여 2 ~ 3 ipynb에 걸쳐 데이터셋 가공
(제가 약간의 혼동이 있어 대부분의 데이터셋 가공 과정이 3번 파일에 있습니다.
2번 파일에서는 데이터 1 가공이 담겨 있습니다)

<수집 기간>

2017년 5월 10일 ~ 2021년 11월 30일 (재임 시작 시기와 일치)

<출처>
1. 네이버 VIEW 검색 엔진(정렬; 오래된순, 기간 지정, 언론사별 크롤링, 키워드: '문재인 사설')

2. [한국 갤럽 국정 수행 지지도](https://www.gallup.co.kr/)
3. [KNU 한국어 감성 사전](http://dilab.kunsan.ac.kr/knusl.html)

### Web Crawling - 언론사별 기사 제목

In [21]:
try:
    Path = "C:/Users/pscq2/Desktop/STUDY/Fourth Course/AI/chromedriver.exe"
    driver = webdriver.Chrome(Path)
    driver.get('https://search.naver.com/search.naver?where=news&query=%EB%AC%B8%EC%9E%AC%EC%9D%B8%20%EC%82%AC%EC%84%A4&sm=tab_opt&sort=2&photo=3&field=0&pd=3&ds=2017.05.10&de=2021.11.30&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20170510to20211130&is_sug_officeid=0')
    print('@: 성공적으로 chromedriver를 실행하였습니다')
except:
    print('@: chromedriver를 실행하는 데 실패하였습니다')

@: 성공적으로 chromedriver를 실행하였습니다


In [22]:
NEWS = [['언론사', '날짜', '기사 제목', '기사 주소']]
press = [['한겨레', 14], ['경향신문', 1], ['국민일보', 2], ['동아일보', 4]] #네이버에서의 언론사별 가나다순 순서 

count = 1
for i in range(len(press)):
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[4]/div/div[1]/a[2]').click() #언론사 분류순
    time.sleep(0.5)
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div/div[2]/div/div/div/ul/li[%d]/a' %press[i][1]).click() #해당 언론사 클릭
    page = 0 #1페이지부터 시작
    print("@: %s 언론사의 기사 크롤링을 시작합니다" %press[i][0])

    for j in range(160):
        # print("@ %d page 크롤링 중 --- " %(page + 1))
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        for k in range(page*10 + 1, page*10 + 11): # 한 페이지에 10개의 기사가 노출
            try:
                news_info = soup.select("#sp_nws%d" %k)[0]
            except: # 마지막 page는 기사가 10개가 아닐 수 있음
                continue
        
            NEWS.append([])
            NEWS[count].append('%s' %press[i][0]) # 언론사 이름을 추가
        
            if len(news_info.find_all('span', 'info')) == 2: #기사 정보를 표현하는 type이 두 가지로 네이버에 표현되어 있으므로 이 부분 분기문으로 해결
                date = news_info.find_all('span', 'info')[1].get_text()
            else:
                date = news_info.find('span', 'info').get_text()
            NEWS[count].append(date) #기사 작성 날짜 추가
        
            title = news_info.find('a', 'news_tit').get_text()
            NEWS[count].append(title) #기사 제목 추가
            
            url = news_info.find('a', 'news_tit')["href"]
            NEWS[count].append(url)
        
            count += 1 
        
        driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/a[2]').click() #다음 페이지 클릭
        page += 1

@: 한겨레 언론사의 기사 크롤링을 시작합니다
@: 경향신문 언론사의 기사 크롤링을 시작합니다
@: 국민일보 언론사의 기사 크롤링을 시작합니다
@: 동아일보 언론사의 기사 크롤링을 시작합니다


In [23]:
NEWS

[['언론사', '날짜', '기사 제목', '기사 주소'],
 ['한겨레',
  '2017.05.10.',
  '‘보수 10년’ 퇴행 바로잡고 ‘미완의 개혁’ 진전시킬 기회',
  'http://www.hani.co.kr/arti/politics/politics_general/794097.html'],
 ['한겨레',
  '2017.05.10.',
  '[한겨레 사설] 문재인 대통령, 국민과 함께 ‘나라다운 나라’ 만들길',
  'http://www.hani.co.kr/arti/opinion/editorial/794106.html'],
 ['한겨레',
  '2017.05.10.',
  '미 언론 “한국 시민혁명” “대북압박 트럼프에 도전”',
  'http://www.hani.co.kr/arti/international/international_general/794200.html'],
 ['한겨레',
  '2017.05.10.',
  '[한겨레 사설] ‘통합과 공존’ 앞세운 문 대통령의 취임사',
  'http://www.hani.co.kr/arti/opinion/editorial/794208.html'],
 ['한겨레',
  '2017.05.10.',
  '[한겨레 사설] ‘이낙연 총리-임종석 실장’ 인선에 거는 기대',
  'http://www.hani.co.kr/arti/opinion/editorial/794219.html'],
 ['한겨레',
  '2017.05.11.',
  '[한겨레 사설] ‘비검찰 민정수석’ 발탁, 검찰개혁의 마지막 기회다',
  'http://www.hani.co.kr/arti/opinion/editorial/794365.html'],
 ['한겨레',
  '2017.05.11.',
  '[한겨레 사설] 조속한 ‘한-미 정상회담’으로 한반도 현안 풀어야',
  'http://www.hani.co.kr/arti/opinion/editorial/794368.html'],
 ['한겨레',
  '2017.05.11.',
  '[한겨레 사설

In [24]:
len(NEWS)

5714

## DataFrame을 활용해 표 만들기

In [25]:
news = pd.DataFrame()
for idx, column_name in enumerate(NEWS[0]):
    news['%s' %column_name] = [NEWS[i][idx] for i in range(1, len(NEWS))]

In [26]:
news

,언론사,날짜,기사 제목,기사 주소
0,한겨레,2017.05.10.,‘보수 10년’ 퇴행 바로잡고 ‘미완의 개혁’ 진전시킬 기회,http://www.hani.co.kr/arti/politics/politics_g...
1,한겨레,2017.05.10.,"[한겨레 사설] 문재인 대통령, 국민과 함께 ‘나라다운 나라’ 만들길",http://www.hani.co.kr/arti/opinion/editorial/7...
2,한겨레,2017.05.10.,미 언론 “한국 시민혁명” “대북압박 트럼프에 도전”,http://www.hani.co.kr/arti/international/inter...
3,한겨레,2017.05.10.,[한겨레 사설] ‘통합과 공존’ 앞세운 문 대통령의 취임사,http://www.hani.co.kr/arti/opinion/editorial/7...
4,한겨레,2017.05.10.,[한겨레 사설] ‘이낙연 총리-임종석 실장’ 인선에 거는 기대,http://www.hani.co.kr/arti/opinion/editorial/7...
...,...,...,...,...
5708,동아일보,2021.11.22.,[사설]‘반값아파트’ ‘기본주택’ ‘역세권첫집’ 모두 현실성 의문이다,https://www.donga.com/news/article/all/2021112...
5709,동아일보,2021.11.24.,"[사설]이재명 노동이사제, 공기업 철밥통만 강화해줄 것",https://www.donga.com/news/article/all/2021112...
5710,동아일보,2021.11.26.,[신문과 놀자!/주니어를 위한 사설 따라잡기]아슬아슬한 ‘위드 코로나’,https://www.donga.com/news/article/all/2021112...
5711,동아일보,2021.11.27.,[사설]李 “탄소세로 30조∼64조” 이러다간 제조업 뿌리 흔들릴 것,https://www.donga.com/news/article/all/2021112...


## 출력 결과를 txt, csv, xlsx 파일로 저장하기

In [27]:
def FileNameCreate(form):
    return os.getcwd() + '/2_Data Prepration.%s' %form

f = open(FileNameCreate('txt'), 'w', encoding = 'UTF-8')
for line in NEWS:
    f.write(str(line) + "\n")
f.close()
news.to_csv(FileNameCreate('csv'), encoding="utf-8-sig",index=False)
news.to_excel(FileNameCreate('xlsx'), index=False)

print("@: %s 위치에 성공적으로 txt, csv, xlsx 파일이 생성되었습니다! " %os.getcwd())

@: C:\Users\pscq2\Desktop\STUDY\Fourth Course\AI\SW_AI\K. Final 위치에 성공적으로 txt, csv, xlsx 파일이 생성되었습니다! 
